In [32]:
!conda install -c ml4t zipline-reloaded

Solving environment: ...working... done

# All requested packages already installed.



UsageError: Cell magic `%%zipline` not found.


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\nickg\anaconda3\envs\liveproject

  added / updated specs:
    - zipline-reloaded


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.3.29  |       haa95532_0         122 KB
    certifi-2021.10.8          |   py38haa95532_2         152 KB
    conda-4.12.0               |   py38haa95532_0        14.5 MB
    openssl-1.1.1n             |       h2bbff1b_0         4.8 MB
    ------------------------------------------------------------
                                           Total:        19.6 MB

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2021.10.~ --> pkgs/main::ca-certificates-2022.3.29-haa95532_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge::certifi-2021.10.8-py38ha~ 

In [50]:
import os
import zipline
os.environ['QUANDL_API_KEY'] = 'aXA3Wj8FvwkVb-5VLnVH'
from zipline.api import order, order_target, record, symbol
from zipline.examples import buyapple as buyapple
import matplotlib.pyplot as plt

In [42]:
!zipline ingest -b quandl

[2022-04-12 19:31:05.479596] INFO: zipline.data.bundles.core: Ingesting quandl.
[2022-04-12 19:31:05.480597] INFO: zipline.data.bundles.quandl: Downloading WIKI metadata.
[2022-04-12 19:31:26.699350] INFO: zipline.data.bundles.quandl: Parsing raw data.
[2022-04-12 19:31:53.705399] INFO: zipline.data.bundles.quandl: Generating asset metadata.
C:\Users\nickg\anaconda3\envs\liveproject\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\nickg\anaconda3\envs\liveproject\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\nickg\anaconda3\envs\liveproject\lib\site-packages\pandas\i

Merging daily equity files:

[2022-04-12 19:33:24.122655] INFO: zipline.data.bundles.core: Ingesting quandl.
[2022-04-12 19:33:24.123656] INFO: zipline.data.bundles.quandl: Downloading WIKI metadata.
[2022-04-12 19:34:00.939712] INFO: zipline.data.bundles.quandl: Parsing raw data.
[2022-04-12 19:34:25.728079] INFO: zipline.data.bundles.quandl: Generating asset metadata.
C:\Users\nickg\anaconda3\envs\liveproject\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\nickg\anaconda3\envs\liveproject\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\nickg\anaconda3\envs\liveproject\lib\site-packages\pandas\i


Merging daily equity files:


In [45]:
def initialize(context):
    context.i = 0
    context.asset = symbol('AAPL')

In [46]:
def handle_data(context, data):
    # Skip first 300 days to get full windows
    context.i += 1
    if context.i < 300:
        return

    # Compute averages
    # data.history() has to be called with the same params
    # from above and returns a pandas dataframe.
    short_mavg = data.history(context.asset, 'price', bar_count=100, frequency="1d").mean()
    long_mavg = data.history(context.asset, 'price', bar_count=300, frequency="1d").mean()

    # Trading logic
    if short_mavg > long_mavg:
        # order_target orders as many shares as needed to
        # achieve the desired number of shares.
        order_target(context.asset, 100)
    elif short_mavg < long_mavg:
        order_target(context.asset, 0)

    # Save values for later inspection
    record(AAPL=data.current(context.asset, 'price'),
           short_mavg=short_mavg,
           long_mavg=long_mavg)


In [47]:
def analyze(context, perf):
    fig = plt.figure()
    ax1 = fig.add_subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax1.set_ylabel('portfolio value in $')

    ax2 = fig.add_subplot(212)
    perf['AAPL'].plot(ax=ax2)
    perf[['short_mavg', 'long_mavg']].plot(ax=ax2)

    perf_trans = perf.loc[[t != [] for t in perf.transactions]]
    buys = perf_trans.loc[[t[0]['amount'] > 0 for t in perf_trans.transactions]]
    sells = perf_trans.loc[
        [t[0]['amount'] < 0 for t in perf_trans.transactions]]
    ax2.plot(buys.index, perf.short_mavg.loc[buys.index],
             '^', markersize=10, color='m')
    ax2.plot(sells.index, perf.short_mavg.loc[sells.index],
             'v', markersize=10, color='k')
    ax2.set_ylabel('price in $')
    plt.legend(loc=0)
    plt.show()

In [52]:
!zipline run -f "C:\Users\nickg\anaconda3\envs\liveproject\Lib\site-packages\zipline\examples\buyapple.py" --start 2016-1-1 --end 2018-1-1 -o buyapple_out.pickle --no-benchmark

Figure(1800x800)


[2022-04-12 21:08:01.261248] INFO: zipline.finance.metrics.tracker: Simulated 503 trading days
first open: 2016-01-04 14:31:00+00:00
last close: 2017-12-29 21:00:00+00:00


In [53]:
import pandas as pd

In [54]:
perf = pd.read_pickle('buyapple_out.pickle')

In [56]:
perf

,period_open,period_close,ending_value,starting_cash,ending_cash,portfolio_value,longs_count,shorts_count,long_value,short_value,...,period_label,algo_volatility,benchmark_period_return,benchmark_volatility,algorithm_period_return,alpha,beta,sharpe,sortino,max_drawdown
2016-01-04 21:00:00+00:00,2016-01-04 14:31:00+00:00,2016-01-04 21:00:00+00:00,0.0,10000000.00,10000000.00,10000000.00,0,0,0.0,0.0,...,2016-01,NaN,0.0,NaN,0.000000e+00,None,None,NaN,NaN,0.000000e+00
2016-01-05 21:00:00+00:00,2016-01-05 14:31:00+00:00,2016-01-05 21:00:00+00:00,1027.1,10000000.00,9998971.90,9999999.00,1,0,1027.1,0.0,...,2016-01,0.000001,0.0,0.0,-1.000000e-07,None,None,-11.224972,-11.224972,-1.000000e-07
2016-01-06 21:00:00+00:00,2016-01-06 14:31:00+00:00,2016-01-06 21:00:00+00:00,2014.0,9998971.90,9997963.90,9999977.90,1,0,2014.0,0.0,...,2016-01,0.000019,0.0,0.0,-2.210000e-06,None,None,-9.823839,-9.588756,-2.210000e-06
2016-01-07 21:00:00+00:00,2016-01-07 14:31:00+00:00,2016-01-07 21:00:00+00:00,2893.5,9997963.90,9996998.40,9999891.90,1,0,2893.5,0.0,...,2016-01,0.000064,0.0,0.0,-1.081000e-05,None,None,-10.592737,-9.688947,-1.081000e-05
2016-01-08 21:00:00+00:00,2016-01-08 14:31:00+00:00,2016-01-08 21:00:00+00:00,3878.4,9996998.40,9996027.80,9999906.20,1,0,3878.4,0.0,...,2016-01,0.000063,0.0,0.0,-9.380000e-06,None,None,-7.511729,-7.519659,-1.081000e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-22 21:00:00+00:00,2017-12-22 14:31:00+00:00,2017-12-22 21:00:00+00:00,871549.8,9367629.16,9365878.06,10237427.86,1,0,871549.8,0.0,...,2017-12,0.007602,0.0,0.0,2.374279e-02,None,None,1.562714,2.498901,-5.682384e-03
2017-12-26 21:00:00+00:00,2017-12-26 14:31:00+00:00,2017-12-26 21:00:00+00:00,851144.3,9365878.06,9364171.36,10215315.66,1,0,851144.3,0.0,...,2017-12,0.007754,0.0,0.0,2.153157e-02,None,None,1.388556,2.157498,-5.682384e-03
2017-12-27 21:00:00+00:00,2017-12-27 14:31:00+00:00,2017-12-27 21:00:00+00:00,853000.0,9364171.36,9362464.36,10215464.36,1,0,853000.0,0.0,...,2017-12,0.007746,0.0,0.0,2.154644e-02,None,None,1.388113,2.156812,-5.682384e-03
2017-12-28 21:00:00+00:00,2017-12-28 14:31:00+00:00,2017-12-28 21:00:00+00:00,857110.8,9362464.36,9360752.56,10217863.36,1,0,857110.8,0.0,...,2017-12,0.007740,0.0,0.0,2.178634e-02,None,None,1.401749,2.178333,-5.682384e-03
